# Adam (Adaptive Moment Estimation)

### 개념 요약
Adam은 현재 가장 널리 사용되는 최적화 알고리즘 중 하나로, RMSprop과 모멘텀(Momentum) 두 가지 방식을 효과적으로 결합한 것임. RMSprop처럼 각 파라미터마다 적응적인 학습률을 계산하여 경사의 크기에 따라 보폭을 조절하고, 모멘텀처럼 이전 경사의 방향을 참고하여 관성을 줌으로써 더 빠르고 안정적으로 최적점에 수렴함.

이름인 'Adam'은 'Adaptive Moment Estimation(적응적 모멘트 추정)'에서 유래했으며, 과거 경사들의 1차 모멘트(평균, 모멘텀)와 2차 모멘트(중심화되지 않은 분산, RMSprop)를 지수 이동 평균하여 사용함.

### 📐 수학적 배경
Adam의 업데이트 과정은 다음과 같이 모멘텀과 RMSprop의 핵심 아이디어를 모두 포함하며, 초기에 모멘트 추정값이 0으로 편향되는 것을 보정하는 단계를 추가함.

1. **경사의 1차 모멘트(Momentum) 업데이트:**
$$ m_t = \beta_1 m_{t-1} + (1 - \beta_1) g_t $$

2. **경사의 2차 모멘트(RMSprop) 업데이트:**
$$ v_t = \beta_2 v_{t-1} + (1 - \beta_2) g_t^2 $$

3. **초기 편향 보정(Bias-correction):**
$$ \hat{m}_t = \frac{m_t}{1 - \beta_1^t} $$
$$ \hat{v}_t = \frac{v_t}{1 - \beta_2^t} $$

4. **파라미터(Weight) 업데이트:**
$$ W_t = W_{t-1} - \eta \frac{\hat{m}_t}{\sqrt{\hat{v}_t} + \epsilon} $$

* $g_t$: 현재 타임스텝 $t$에서의 경사
* $m_t, v_t$: 각각 경사의 1차, 2차 모멘트 벡터
* $\hat{m}_t, \hat{v}_t$: 편향이 보정된 모멘트 벡터
* $\beta_1, \beta_2$: 모멘트 추정을 위한 지수 감쇠율 (보통 0.9, 0.999 사용)
* $\eta$: 학습률(learning rate)
* $\epsilon$: 분모가 0이 되는 것을 방지하기 위한 작은 상수 (예: $10^{-8}$)

### 라이브러리 임포트 (Setup)

In [ ]:
# !pip install numpy matplotlib

import numpy as np
import matplotlib.pyplot as plt

### 예제 데이터 준비 (Data Preparation)
Adam의 성능을 확인하기 위해, RMSprop 예제와 동일하게 각 차원별 곡률이 다른 손실 함수 $f(x, y) = 0.1x^2 + 5y^2$를 사용함. Adam이 이 비등방성(anisotropic) 손실 함수 표면을 얼마나 효율적으로 탐색하는지 확인함.

In [ ]:
# 손실 함수 정의
def loss_function(w):
    return 0.1 * w[0]**2 + 5 * w[1]**2

# 손실 함수의 경사(미분) 정의
def gradient(w):
    return np.array([0.2 * w[0], 10 * w[1]])

### 핵심 로직 구현 (Code Implementation)
Adam 알고리즘을 단계별로 직접 구현하여 파라미터 업데이트 과정을 살펴봄.

In [ ]:
# 하이퍼파라미터 설정 (논문에서 제안된 기본값)
learning_rate = 0.3
beta1 = 0.9
beta2 = 0.999
epsilon = 1e-8
n_iterations = 50

# 파라미터 및 모멘트 벡터 초기화
w = np.array([-10.0, 1.5])
m = np.zeros_like(w)  # 1차 모멘트 벡터
v = np.zeros_like(w)  # 2차 모멘트 벡터
w_history = [w.copy()]

print("--- Adam 최적화 과정 ---")
for t in range(1, n_iterations + 1):
    # 1. 현재 위치에서 경사 계산
    grad = gradient(w)
    
    # 2. 1차 모멘트(모멘텀) 업데이트
    m = beta1 * m + (1 - beta1) * grad
    
    # 3. 2차 모멘트(RMSprop) 업데이트
    v = beta2 * v + (1 - beta2) * (grad**2)
    
    # 4. 편향 보정
    m_hat = m / (1 - beta1**t)
    v_hat = v / (1 - beta2**t)
    
    # 5. 파라미터 업데이트
    w = w - learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)
    w_history.append(w.copy())

    if t % 5 == 0:
        # 핵심 변수 확인: 편향 보정된 m_hat과 v_hat이 실제 업데이트에 어떻게 기여하는지 확인
        print(f"Iter {t:2d}: m_hat=[{m_hat[0]:6.3f}, {m_hat[1]:6.3f}], v_hat=[{v_hat[0]:6.3f}, {v_hat[1]:6.3f}], W=[{w[0]:6.3f}, {w[1]:6.3f}]")

### 📊 결과 확인 및 시각화 (Results & Visualization)
Adam을 이용한 파라미터 업데이트 경로를 등고선 플롯 위에 시각화함. 모멘텀의 관성과 RMSprop의 적응적 학습률 덕분에 진동이 적고, 빠르면서도 안정적으로 최저점을 향해 이동하는 것을 확인할 수 있음.

In [ ]:
w_history = np.array(w_history)

# 등고선 플롯을 위한 그리드 생성
x_mesh, y_mesh = np.meshgrid(np.linspace(-12, 12, 100), np.linspace(-2, 2, 100))
z_mesh = loss_function(np.array([x_mesh, y_mesh]))

plt.figure(figsize=(10, 7))
plt.contourf(x_mesh, y_mesh, z_mesh, levels=20, cmap='viridis_r', alpha=0.7)
plt.colorbar(label='Loss')

# 파라미터 업데이트 경로 시각화
plt.plot(w_history[:, 0], w_history[:, 1], 'o-', color='red', label='Adam Path')

plt.title('Adam Optimization Path', fontsize=16)
plt.xlabel('Parameter w_1')
plt.ylabel('Parameter w_2')
plt.legend()
plt.grid(True)
plt.axis('equal')
plt.show()

### 고려사항 (Considerations)

* <b>기본 하이퍼파라미터의 효율성</b>: Adam 논문에서 제안된 기본값($\eta=0.001, \beta_1=0.9, \beta_2=0.999$)은 다양한 문제에서 좋은 성능을 보여주어, 별도의 튜닝 없이도 baseline으로 널리 사용됨.
* **AdamW**: Adam의 가중치 감쇠(weight decay) 방식을 개선한 AdamW라는 변형이 있음. L2 정규화와 가중치 감쇠가 동일하지 않다는 점에서 착안하여, Adam의 업데이트 식에서 가중치 감쇠 항을 분리한 것으로 더 나은 일반화 성능을 보이는 것으로 알려져 있음.

### 최종 요약 (Conclusion)
Adam은 모멘텀과 RMSprop의 장점을 결합한 매우 강력하고 효율적인 최적화 알고리즘임. 과거 경사 정보(모멘텀)를 통해 전진할 방향에 대한 관성을 얻고, 최근 경사 크기 정보(RMSprop)를 통해 파라미터별로 학습률을 동적으로 조절함.

이러한 복합적인 접근 방식 덕분에 Adam은 손실 함수 공간이 복잡하고, 노이즈가 많으며, 경사가 희소한 환경에서도 안정적으로 잘 작동함. 이러한 강점 때문에 Adam은 오늘날 수많은 딥러닝 모델의 기본 최적화 도구로 자리 잡았음.